## Image Data Analysis

### Read example image with cv2

In [26]:
from cv2 import imdecode, IMREAD_UNCHANGED, imshow, waitKey, destroyAllWindows, imwrite
import json
import requests
import numpy as np

all_data = json.load(open('./merged.json'))

def get_image_data(img_url):
    print(f'GET {ndex_num}:{img_url}')
    raw_image_data = requests.get(img_url, stream=True).raw
    image_array = np.asarray(bytearray(raw_image_data.read()), dtype="uint8")
    image_data = imdecode(image_array, IMREAD_UNCHANGED)
    ext = img_url.split('.')[-1].lower().strip()
    return image_data.shape


In [27]:
height, width, num_channels = get_image_data(all_data['001']['sprites'][0])
(height, width, num_channels)

GET 001:https://archives.bulbagarden.net/media/upload/thumb/4/48/Build-A-Bear_PokemonPartyTShirt.png/120px-Build-A-Bear_PokemonPartyTShirt.png


(120, 120, 3)

In [29]:
for ndex in list(all_data.keys())[:3]:
    sprites = all_data[ndex]['sprites']
    for sprite_url in sprites[:5]:
        print(get_image_data(sprite_url))

GET 001:https://archives.bulbagarden.net/media/upload/thumb/4/48/Build-A-Bear_PokemonPartyTShirt.png/120px-Build-A-Bear_PokemonPartyTShirt.png
(120, 120, 3)
GET 001:https://archives.bulbagarden.net/media/upload/3/38/Spr_b_2c_001_s.png
(56, 56, 4)
GET 001:https://archives.bulbagarden.net/media/upload/b/b6/Spr_1y_001.png
(56, 56, 4)
GET 001:https://archives.bulbagarden.net/media/upload/7/75/001_s.png
(96, 96, 4)
GET 001:https://archives.bulbagarden.net/media/upload/e/ee/001MSStad.png
(40, 40, 4)
GET 001:https://archives.bulbagarden.net/media/upload/thumb/1/11/Ivysaur_04_018.png/120px-Ivysaur_04_018.png
(120, 120, 4)
GET 001:https://archives.bulbagarden.net/media/upload/d/dd/Menu_Colo_002.png
(44, 44, 4)
GET 001:https://archives.bulbagarden.net/media/upload/thumb/c/c1/002Ivysaur_RB.png/120px-002Ivysaur_RB.png
(109, 120, 4)
GET 001:https://archives.bulbagarden.net/media/upload/thumb/e/e4/Spr_b_7s_002_s.png/120px-Spr_b_7s_002_s.png
(120, 120, 4)
GET 001:https://archives.bulbagarden.net/medi